In [ ]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model

In [ ]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [ ]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [ ]:
from federatedscope.contrib.trainer.laplacian_with_clustering_trainer import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

### Register metric

### Create the config

#### Set data, model, trainer and metric

In [ ]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup_own.yaml")
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
#cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']

cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance=1e3
cfg.params.eps=1e-15
cfg.params.p=0.
cfg.params.EPS_1 = 0.1
cfg.params.EPS_2 = 0.01
cfg.params.cluster_start = 3000

#### configure other options

In [ ]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = 'Laplacian_clustering_batch_class_clients_corrected_batch_orig_lr_csd_1e3'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 20000

cfg.federate.client_num = 9
cfg.early_stop.patience = 20000
#cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0005
cfg.grad.grad_clip = 5.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [ ]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(False)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [ ]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup_per_client_class_all_same_lr.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [ ]:
from federatedscope.contrib.workers.laplacian_with_clustering_client import LaplacianClusteringClient
from federatedscope.contrib.workers.laplacian_with_clustering_server import LaplacianWithClusteringServer

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


In [ ]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianWithClusteringServer,
                       client_class=LaplacianClusteringClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

In [ ]:
data

In [ ]:
import os

In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']